This is the sample code to check the environment for running GluonTS.

Before running sample.py, please check requirement.txt and install required packages.

Sample.py includes two deep learning models, DeepAR and MQCNN.

Papers about two models for time series forecasting:

DeepAR: https://arxiv.org/pdf/1704.04110.pdf

MQCNN: https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_conv/ (wavenet)

In [15]:
from functools import partial

import pandas as pd

import sys
sys.path.insert(0,'/home/chenkai/Documents/git_projects/gluon-ts/src')


from gluonts.dataset.repository.datasets import get_dataset
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.trainer import Trainer

In [16]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQCNNEstimator

In [17]:
epochs = 5
num_batches_per_epoch = 10
dataset_name = "m4_hourly"
dataset = get_dataset(dataset_name)

In [19]:
results = []
# If you want to use GPU, please set ctx="gpu(0)"　/ ctx="cpu"
estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            ctx="cpu",
            epochs=epochs,
            num_batches_per_epoch=num_batches_per_epoch
        )
    ),
    partial(
        MQCNNEstimator,
        trainer=Trainer(
            ctx="cpu",
            epochs=epochs,
            num_batches_per_epoch=num_batches_per_epoch
        )
    ),
]

for estimator in estimators:
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq
    )
    predictor = estimator.train(dataset.train)
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    results.append(eval_dict)

df = pd.DataFrame(results)
sub_df = df[
    [
        "dataset",
        "estimator",
        "mean_wQuantileLoss",
    ]
]
print(sub_df)

  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


Running evaluation: 100%|██████████| 414/414 [00:01<00:00, 391.08it/s]


     dataset        estimator  mean_wQuantileLoss
0  m4_hourly  DeepAREstimator            0.084096
1  m4_hourly   MQCNNEstimator            0.425345


In [21]:
df.columns

Index(['MSE', 'abs_error', 'abs_target_sum', 'abs_target_mean',
       'seasonal_error', 'MASE', 'MAPE', 'sMAPE', 'OWA', 'MSIS',
       'QuantileLoss[0.1]', 'Coverage[0.1]', 'QuantileLoss[0.2]',
       'Coverage[0.2]', 'QuantileLoss[0.3]', 'Coverage[0.3]',
       'QuantileLoss[0.4]', 'Coverage[0.4]', 'QuantileLoss[0.5]',
       'Coverage[0.5]', 'QuantileLoss[0.6]', 'Coverage[0.6]',
       'QuantileLoss[0.7]', 'Coverage[0.7]', 'QuantileLoss[0.8]',
       'Coverage[0.8]', 'QuantileLoss[0.9]', 'Coverage[0.9]', 'RMSE', 'NRMSE',
       'ND', 'wQuantileLoss[0.1]', 'wQuantileLoss[0.2]', 'wQuantileLoss[0.3]',
       'wQuantileLoss[0.4]', 'wQuantileLoss[0.5]', 'wQuantileLoss[0.6]',
       'wQuantileLoss[0.7]', 'wQuantileLoss[0.8]', 'wQuantileLoss[0.9]',
       'mean_wQuantileLoss', 'MAE_Coverage', 'dataset', 'estimator'],
      dtype='object')

In [22]:
sub_df = df[
    [
        "dataset",
        "estimator",
        "MSE",
        "RMSE"
    ]
]

sub_df

,dataset,estimator,MSE,RMSE
0,m4_hourly,DeepAREstimator,2.062212e+07,4541.158042
1,m4_hourly,MQCNNEstimator,NaN,NaN
